#Table of Contents
* [Notes!](#Notes!)
* [App](#App)
* [Configuration](#Configuration)
* [Initialization](#Initialization)
* [Title](#Title)
* [Survey](#Survey)
* [Reality check](#Reality-check)
* [Goal](#Goal)
	* [Cowboy](#Cowboy)
	* [workflow](#workflow)
* [Docker](#Docker)
	* [Hello, world](#Hello,-world)
	* [/etc/issue](#/etc/issue)
	* [get_ext_ip](#get_ext_ip)
		* [Don't conflate concepts.](#Don't-conflate-concepts.)
		* [via commit](#via-commit)
		* [image size check (commit)](#image-size-check-%28commit%29)
		* [via dockerfile](#via-dockerfile)
	* [Review](#Review)
* [Our Host](#Our-Host)
* [Our App](#Our-App)
	* [Nothing up my sleeve](#Nothing-up-my-sleeve)
	* [Getting to know your web server](#Getting-to-know-your-web-server)
		* [Our web server's code](#Our-web-server's-code)
		* [Our web server's requirements file](#Our-web-server's-requirements-file)
		* [Our web server's **dockerfile**](#Our-web-server's-**dockerfile**)
		* [Our web server's **image**](#Our-web-server's-**image**)
* [Docker Compose](#Docker-Compose)
	* [flask in yaml](#flask-in-yaml)
	* [flask & redis in yaml](#flask-&-redis-in-yaml)
	* [reordering our dockerfile](#reordering-our-dockerfile)
	* [decoupling web.py and our web **image**](#decoupling-web.py-and-our-web-**image**)
	* [decoupling web.py and web_image](#decoupling-web.py-and-web_image)
	* [Don't expect too much from **docker-compose up**.](#Don't-expect-too-much-from-**docker-compose-up**.)
	* [Don't put any more in your image than you must.](#Don't-put-any-more-in-your-image-than-you-must.)
	* [Don't forget volumes.](#Don't-forget-volumes.)
* [Amazon Web Services](#Amazon-Web-Services)
	* [TLAs (Three Letter Acronyms)](#TLAs-%28Three-Letter-Acronyms%29)


# Notes!


Don't forget volumes.
  **docker rm** does _not_ remove associated volumes.
  **docker-compose** replaces existing containers.
  orphaned volumes add up.
  Use **docker rm** -v or --volume to remove the volume.
  Ticket?

Don't leave any initialization out of your image.
  Configuration files can live in the source.

Don't wait on downloads.
  Circle-CI too ephemeral.
    circle.yml
    caching tar hack
  
Don't wait on builds.
  Don't redeploy in development.
  Use a mounted source volume.

Don't complicate early.
  Amazon ECS, while powerful, can wait.
  As of last week, ECS will use docker-compose.

Don't lose yourself.
  docker in docker
      Let's imagine what deploying to our production server will entail.
    I expect we'll need ssh access to it in order to run commands.
    It will have docker to run our containers and docker-compose to coordinate them.
    Oh, our code! How do we get it from the repo onto this server?
        git pull?
            ew. That feels wrong.
        Didn't we just learn how to build Dockerfiles?
            Let's bake our code into an image.
            The docker registry can house our production-ready image.
            Then a "deploy" becomes **docker run**.
            Wait. I mean **docker compose***.
            Wait. **docker compose*** requires a yml file.
            That lives in our repo. Crap.
            How do I get my docker-compose.yml onto this server?
            Well, I could put the docker-compose.yml into an image...
            Then, I could have a "deploy" container.
            It could run "**docker-compose** run" for me.
   So, our target deploy process: 
    - ssh to server that hase docker and docker-compose.
    - **docker run** deploy:commit-hash
    - smile
   I actually got this to run. (shame/pride)
   In the process, I learned about aufs (used by docker for image layers).
   The aufs layers don't live inside the container.
   They live on the host file system. 
   How to get around this?
   **docker export** lets you turn an image into a regular tar file.
   My build process involved exporting my images before committing my deploy container.
   This took forever, used _way_ too much space, and felt awful.
   So I went online, asked for help, and found I'd gone WAY off the beaten path.
   The "docker" way: **docker-machine**.
   **docker-machine create** takes a --driver command.


Don't repeat yourself.
  docker-compose.yml
    develop.yml
    test.yml
    deploy.yml
    
    Remarkably similar with many identical sections, but a few key differences.
    How do we keep the shared sections DRY?
    DRY (Don't Repeat Yourself)
    
    Must we bust out jinja2 templating?
    
    Great news: **docker compose** provides a solution!
        extends:
          file:
          service:
    
    develop.yml >-+-> deploy.yml >-+-> app/service.yml
    test.yml    >-/                \-> common/service.yml
    
    Hooray!
    
    Sad news: container-transform has no idea knowledge of 'extends'.
    So, now we can't deploy to ecs.
    Let's get simpler. Let's just deploy to our own EC2 instance.
    
    Whatever. We didn't really need any of that complication/functionality yet anyway.
    
    We learned how to spin up EC2 instances from Launch Configurations.
    
    
Don't duplicate tools.
   docker-machine replaced Boot2Docker.
   KiteMatic exists.
    
continuous integration tools
    travis
    jenkins
    buildbot
    shippable
    circle-ci
    
Don't get lost.
  bash prompt hacking dm:, d:, dc:
 
Don't share secrets.
  not in the repo
  
local, test, qa, production
informatics, arcus, converge
customer_a, customer_b, customer_c


# App

# Configuration

In [ ]:
%%javascript
IPython.load_extensions('calico-document-tools');

In [ ]:
from IPython.html.services.config import ConfigManager
from IPython.utils.path import locate_profile
cm = ConfigManager(profile_dir=locate_profile(get_ipython().profile))
cm.update('livereveal', dict(
    theme='white',
    transition='none',
    start_slideshow_at='selected',
    width=1024,
    height=768,
))

# Initialization

In [ ]:
# Presentation Initialization 
docker_stop_all = 'docker stop $(docker ps -a -q)'
docker_remove_all = 'docker rm $(docker ps -a -q)'
docker_nuke = docker_stop_all + '&&' + docker_remove_all
!$docker_nuke
!rm Dockerfile
!rm -rf /ffdd/app
!mkdir /ffdd/app

In [ ]:
# Pull images for our app
!docker pull busybox:buildroot-2013.08.1
!docker pull python:3.4
!docker pull redis:3.0.2

# Title

Fumbling Forward
========
Docker Don'ts
=======
Jason M. Green

gJigsaw  <sub><sub>gMail, GitHub, <sub> and **very** infrequently Twitter</sub></sub></sub>

# Survey

<img style="float: right; margin: 0px 15px 15px 0px;" 
src="files/images/no-chair.png" 
height="200" width="200" />
All Rise

<img style="float: right; margin: 0px 15px 15px 0px;" 
src="files/images/no-chair.png" 
height="200" width="200" />
All Rise
<p>
<sub>This will only take a moment.</sub>


Don't sit down if you have...

- heard of Docker

- installed Docker

- run a Docker container

- written a Dockerfile

- run docker-compose

Don't sit down if you have...
- run tests in a docker container

- setup an on-commit action

- run docker-machine

- consumed CI-as-a-service

- provided CI-as-a-service

Don't sit down if you have...
- desire to present in my stead

<sub> <sub> Please? </sub> </sub> 

# Reality check

I claim neither authority nor expertise.

This talk may include terrible advice.

Please speak up; I'd love to improve it.

# Goal

<img style="float: right; margin: 0px 15px 15px 0px;" 
src="files/images/holy_grail.jpg" 
height="200" width="200" />

Our Quest
=====
**Continuous Delivery**

 
<p>
 

<img style="float: right; margin: 0px 15px 15px 0px;" 
src="files/images/holy_grail.jpg" 
height="200" width="200" />

Our Quest
=====
**Continuous Delivery**

Wait. 
<p>
What does that mean?

<img style="float: right; margin: 0px 15px 15px 0px;" 
src="files/images/holy_grail.jpg" 
height="200" width="200" />

Our Quest
=====
**Continuous Delivery**

<sub>"is the practice of releasing every good build to users"</sub>
<p><t><sub><sub> - [Jez Humble](http://www.amazon.com/Jez-Humble/e/B003SNGS8E/)
(VP at Chef and author of Continuous Delivery)</sub></sub>

Shortest possible path
---------------------

- imagine
 - Have an idea.

- code 
 - Write/Edit files.

- deploy
 - Get it out there.

Wait - we don't need CD tools for this!

## Cowboy

<img style="float: right; margin: 0px 15px 15px 0px;" 
src="files/images/cowboy.jpg" 
height="200" width="300" />
Code in production!
-------------------
- imagine
  - I've a great idea.
- code
  - ssh web_server.example.com
  - nano server.py
- Deploy
  - sudo shutdown -r now

You may have experienced this.

<sub><sub>It haunts my dreams.</sub></sub>

What do we need to add?

Version Control, Testing, Code Review 

## workflow

<img src="files/images/targetted_flow.png" height="200" width="800" />

Enough daydreaming. Let's talk tools.
<img style="float: right; margin: 0px 15px 15px 0px;" 
src="files/images/toolbox.png" 
height="200" width="300" />

<!-- Other Tools -->
<img src="files/images/not-appearing.jpg" height="200" width="800" />


# Docker

<img style="float: left; margin: 0px 15px 15px 0px;" 
src="files/images/docker.png"
height="400" width="400" />
<img style="float: right; margin: 0px 15px 15px 0px;" 
src="files/images/docker_star.jpg"
height="400" width="400" />

Applications can own their software and OS dependencies.

Docker
------

Don't conflate these concepts:

**Dockerfile**s, **image**s, & **container**s.

<img style="float: right; margin: 0px 15px 15px 0px;" 
src="files/images/George-Takei-oh-my.jpg" 
height="200" width="300" />

Grok this!

<img src="files/images/docker_overview.png"/>

## Hello, world

A conical dive-right-in beginning.

In [1]:
# Locally
!echo Hello, world.

Hello, world.


In [2]:
# In a docker container
!docker run ubuntu echo Hello, world.

Hello, world.


Let's split it into pieces.

In [3]:
echo_command = "echo Hello, world!"
image = 'ubuntu'
!docker run $image $echo_command

Hello, world!


Ubuntu represents just one example **image**.

Over 15,000 different **image**s exist.

Let's look at a few common **base image**s.

In [4]:
some_base_images = ['fedora', 'ubuntu', 'debian', 'centos', 'busybox']

We can easily **pull** images from hub.docker.com using **docker pull**.

In [5]:
!docker pull busybox

latest: Pulling from busybox



Digest: sha256:38a203e1986cf79639cfb9b2e1d6e773de84002feea2d4eb006b52004ee8502d
Status: Image is up to date for busybox:latest


We needn't _explicitly_ **pull** an **image** before **run**ning it.

Let's run our echo command in a few different **container**s.

In [6]:
for image in some_base_images:
    !docker run $image $echo_command

Hello, world!
Hello, world!
Hello, world!
Hello, world!
Hello, world!


Boring - they all look the same.

## /etc/issue

Let's see the differences between these containers.

In [7]:
for image in some_base_images:
    print(image + ': ', end='')
    issue = !docker run $image cat /etc/issue
    print(''.join(issue))

fedora: Fedora release 22 (Twenty Two)Kernel \r on an \m (\l)
ubuntu: Ubuntu 14.04.2 LTS \n \l
debian: Debian GNU/Linux 8 \n \l
centos: \SKernel \r on an \m
busybox: Welcome to Buildroot


Better, but still just an echo.

## get_ext_ip

Let's find out our external IP Address.

First, define a command and try it locally.

In [8]:
#get_ext_ip = 'curl ifconfig.me' # Skipping due to Slow WIFI!
get_ext_ip = 'curl --help | grep cookie'
!$get_ext_ip

 -b, --cookie STRING/FILE  String or file to read cookies from (H)
 -c, --cookie-jar FILE  Write cookies to this file after operation (H)
 -j, --junk-session-cookies Ignore session cookies read from file (H)


Now let's try it inside a docker container.

In [9]:
image = 'ubuntu'
!docker run $image $get_ext_ip

exec: "curl": executable file not found in $PATH
Error response from daemon: Cannot start container 9384fdf0f3283ed897d357ea2714f0ac2a91135cd00651a96fe350eb6f75e36d: [8] System error: exec: "curl": executable file not found in $PATH


Why didn't this work?

Our first failed dependency! <p> We don't have curl installed within our **container**.

So - let's install it!

### Don't conflate concepts.

Our **container** runs Ubuntu and Ubuntu uses apt-get.

We know how to run commands in our **container**.

So let's just do that.

In [10]:
image = 'ubuntu:latest'
install_curl = 'apt-get install -y curl'
container = 'my_container'
!docker run --name $container $image $install_curl

Reading package lists...
Building dependency tree...
Reading state information...
The following extra packages will be installed:
  ca-certificates krb5-locales libasn1-8-heimdal libcurl3 libgssapi-krb5-2
  libgssapi3-heimdal libhcrypto4-heimdal libheimbase1-heimdal
  libheimntlm0-heimdal libhx509-5-heimdal libidn11 libk5crypto3 libkeyutils1
  libkrb5-26-heimdal libkrb5-3 libkrb5support0 libldap-2.4-2
  libroken18-heimdal librtmp0 libsasl2-2 libsasl2-modules libsasl2-modules-db
  libwind0-heimdal openssl
Suggested packages:
  krb5-doc krb5-user libsasl2-modules-otp libsasl2-modules-ldap
  libsasl2-modules-sql libsasl2-modules-gssapi-mit
  libsasl2-modules-gssapi-heimdal
The following NEW packages will be installed:
  ca-certificates curl krb5-locales libasn1-8-heimdal libcurl3
  libgssapi-krb5-2 libgssapi3-heimdal libhcrypto4-heimdal libheimbase1-heimdal
  libheimntlm0-heimdal libhx509-5-heimdal libidn11 libk5crypto3 libkeyutils1
  libkrb5-26-heimdal libkrb5-3 libkrb5support0 libldap-2

Now we can try to run it again.

In [11]:
image = 'ubuntu'
!docker run $image $get_ext_ip

exec: "curl": executable file not found in $PATH
Error response from daemon: Cannot start container 788079e717a83550fb468e302bcc58dd9541d9c796f1ff33a5cc643e67e8096f: [8] System error: exec: "curl": executable file not found in $PATH


What gives?

Don't conflate **container**s & **image**s.

**docker run** creates a **container** from an **image**.

So let's build an **image**!

We've 2 ways to build an **image**:
 - via **docker commit**
 - via **Dockerfile**

We'll examine both.

### via commit

**docker commit** turns a **container** into an **image**.

So we'll need a **container**.

We've started a few containers.
<p>
We can see them via **docker ps**.

In [12]:
!docker ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS                NAMES
9d3ac7a7309f        web:20150629-1800   "/app/web.py"       20 minutes ago      Up 20 minutes       0.0.0.0:80->80/tcp   app_web_1           


Why don't we see any?

Don't forget '-a'.

By default **docker ps** only shows _running_ containers.
<p>
We can add -a to see _all_ containers.

In [13]:
# show _all_ containers.
!docker ps -a

CONTAINER ID        IMAGE               COMMAND                CREATED              STATUS                          PORTS                NAMES
788079e717a8        ubuntu              "curl --help"          About a minute ago                                                        sleepy_heisenberg    
f052cfb8f121        ubuntu:latest       "apt-get install -y    About a minute ago   Exited (0) About a minute ago                        my_container         
9384fdf0f328        ubuntu              "curl --help"          About a minute ago                                                        sick_sammet          
139bc48328a0        busybox             "cat /etc/issue"       2 minutes ago        Exited (0) 2 minutes ago                             ecstatic_rosalind    
2c48b35c6d94        centos              "cat /etc/issue"       2 minutes ago        Exited (0) 2 minutes ago                             distracted_meitner   
bf0d4e68ba92        debian              "cat /etc/issue"      

We named our container earlier.

In [14]:
container

'my_container'

In [15]:
# show only our container.
!docker ps -a | grep $container

f052cfb8f121        ubuntu:latest       "apt-get install -y    About a minute ago   Exited (0) About a minute ago                        my_container         


Let's **docker commit** our **container** into an **image**. 

In [16]:
image = 'ubuntu_with_curl_via_commit'
!docker commit $container $image

49431787dad2a54bb94db865f0383ce09785ae658c7968db254c4d7383dc751b


**docker commit** outputs the ID of the **image** it created. <p>We'll ignore it and just use the name we assigned to it.

As we've finished with them, let's clean up all our containers.

In [17]:
# stop all containers
!docker stop $(docker ps -a -q)
# remove all containers
!docker rm $(docker ps -a -q)

788079e717a8
f052cfb8f121
9384fdf0f328
139bc48328a0
2c48b35c6d94
bf0d4e68ba92
a24e08991e35
cd5dbb9fe2f7
6c2a0a8b95d7
898757a22487
6fb746ff4889
d40cf6702b76
2bf6c989017a
252b9ee1f690
cc0cb8d3c362
9d3ac7a7309f
788079e717a8
f052cfb8f121
9384fdf0f328
139bc48328a0
2c48b35c6d94
bf0d4e68ba92
a24e08991e35
cd5dbb9fe2f7
6c2a0a8b95d7
898757a22487
6fb746ff4889
d40cf6702b76
2bf6c989017a
252b9ee1f690
cc0cb8d3c362
9d3ac7a7309f


### image size check (commit)

Let's review the image we just created via **docker commit**...

In [18]:
!docker images $image

REPOSITORY                    TAG                 IMAGE ID            CREATED             VIRTUAL SIZE
ubuntu_with_curl_via_commit   latest              49431787dad2        32 seconds ago      199.7 MB


...and compare it to the ubuntu **base image**...

In [19]:
!docker images ubuntu | grep latest

ubuntu              latest              6d4946999d4f        7 weeks ago         188.3 MB


Installing curl grew our image by ~11 MB.

We've **commit**ted a **container** into **an image**. 
<p>
Let's try our command in a **container** built from this new **image**.

In [20]:
!docker run $image $get_ext_ip

 -b, --cookie STRING/FILE  String or file to read cookies from (H)
 -c, --cookie-jar FILE  Write cookies to this file after operation (H)
 -j, --junk-session-cookies Ignore session cookies read from file (H)


What caused the extra noise?

Any questions on **docker commit** before we try it again via **Dockerfile**?

### via dockerfile

A **Dockerfile** extends a **base image**.

In [21]:
ubuget_dockerfile = """
FROM ubuntu:15.10
RUN apt-get install -y curl
"""
!echo "$ubuget_dockerfile" > Dockerfile
!cat Dockerfile


FROM ubuntu:15.10
RUN apt-get install -y curl



This **Dockerfile** extends the *Ubuntu:15.10* image and only performs one step: install curl.

Now that we've a **Dockerfile**, we can **build** an **image**.

In [22]:
image = 'ubtuntu_with_curl_via_dockerfile'
!docker build --tag $image . | tail -n 5

The command '/bin/sh -c apt-get install -y curl' returned a non-zero code: 100

E: Failed to fetch http://archive.ubuntu.com/ubuntu/pool/main/c/curl/curl_7.38.0-3ubuntu3_amd64.deb  404  Not Found [IP: 91.189.92.201 80]

E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?
Fetched 1548 kB in 13s (114 kB/s)


Why did this one fail?

Don't forget remote installation dependencies.

Our image needs an update.

In [23]:
dockerfile = """
FROM ubuntu:15.10
RUN apt-get update -y
RUN apt-get install -y curl
"""
!echo "$dockerfile" > Dockerfile
!time docker build --no-cache --tag $image . | tail -n 3

 ---> cba61456b184
Removing intermediate container 187cef85390f
Successfully built cba61456b184

real	0m47.418s
user	0m0.242s
sys	0m0.035s


Note the time.

Let's try our command again, using our newly built **image**.

In [24]:
!docker run $image $get_ext_ip

 -b, --cookie STRING/FILE  Read cookies from STRING/FILE (H)
 -c, --cookie-jar FILE  Write cookies to FILE after operation (H)
 -j, --junk-session-cookies  Ignore session cookies read from file (H)


## Review

A quick review of some **docker** commands:
 - **docker build** (an **image** from a **Dockerfile**)
 - **docker commit** (an **image** from a **container**)
 - **docker run** (a **container** from an **image**)

**image** specific:
 - **docker images**: list **image**s
 - **docker pull**: fetch **image**s

**container** specific:
 - **docker ps**: list (running) **container**s
 - **docker stop**: stop **container**s
 - **docker rm**: delete stopped **container**s

Any basic **Docker** questions before we continue?

# Our Host

local

In [25]:
# create new docker host
# docker-machine create --driver virtualbox FOO

EC2 instance

In [ ]:
docker_machine_arguments = """
    --driver  -iam-instance-profile ecsInstanceRole \
    --amazonec2-security-group sg-12345678 \
    --amazonec2-vpc-id vpc-12345678 \
    --amazonec2-subnet-id subnet-12345678 \
    --amazonec2-region us-east-1 \
    --amazonec2-zone a \
    --amazonec2-instance-type t2.micro \
    --amazonec2-root-size 25 \
"""
machine_name = 'my_host'
#!docker-machine create $docker_machine_arguments $machine_name

Directing docker-machine and docker

In [ ]:
# point docker-machine to new machine
# docker-machine active FOO

In [26]:
# point docker to new machine
# eval "$(docker-machine env ffdd)"

# Our App

## Nothing up my sleeve

In [27]:
!rm -rf /ffdd/app/
!mkdir /ffdd/app/
!ls -al /ffdd/app

total 0
drwxr-xr-x   2 jasongreen  staff   68 Aug  2 14:20 .
drwxr-xr-x  15 jasongreen  staff  510 Aug  2 14:20 ..


## Getting to know your web server

###Our web server's code

In [28]:
web_py = """#!/usr/bin/env python

from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello():
    return 'Hello, world!'
    
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=80, debug=True)
"""
!echo "$web_py" > /ffdd/app/web.py
!chmod +x /ffdd/app/web.py

###Our web server's requirements file

In [29]:
web_requirements_txt = """
Flask==0.10.1
"""
!echo "$web_requirements_txt" > /ffdd/app/requirements.txt

###Our web server's **dockerfile**

In [30]:
web_dockerfile = """
FROM python:3.4

RUN mkdir /app
ADD web.py /app/web.py

ADD requirements.txt /requirements.txt

RUN pip install -r /requirements.txt

EXPOSE 80
"""
!echo "$web_dockerfile" > /ffdd/app/web_dockerfile

###Our web server's **image**

In [31]:
web_image = "web:20150629-1800"
!docker build --tag $web_image -f /ffdd/app/web_dockerfile /ffdd/app | tail -n 3  

 ---> 8c7ecfe666e0
Removing intermediate container a7c65fa451b1
Successfully built 8c7ecfe666e0


Our web server's **container**

In [32]:
!docker run -d -p "80:80" $web_image /app/web.py

dfae19ce8dabbe52ce29e4269990720a47af3bed14e7b8dacbd8680d33f905d8


Needs more ~~cowbell~~ **container**s!

# Docker Compose

<img style="float: right; margin: 0px 15px 15px 0px;" 
src="files/images/docker_compose.png"
height="400" width="400" />

Docker Compose uses a YAML file to coordinate containers.

So let's add one.

## flask in yaml

In [33]:
app_yml = """
web:
  image: web:20150629-1800
  ports:
    - '80:80'
  command: /app/web.py
"""
!echo "$app_yml" > /ffdd/app/app.yml 

Now, let's start our app.

In [34]:
!docker-compose -f /ffdd/app/app.yml up -d 

/usr/local/Cellar/fig/1.3.3/libexec/vendor/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
Creating app_web_1...
Cannot start container b591926c3be8d82d0d362a7c252ccd5e1d43419d5360a8d40b250aa4869e4a71: Bind for 0.0.0.0:80 failed: port is already allocated


Let's shut down our previous web server.

In [35]:
docker_stop_all = 'docker stop $(docker ps -a -q)'
docker_remove_all = 'docker rm $(docker ps -a -q)'
docker_nuke = docker_stop_all + '&&' + docker_remove_all
!$docker_nuke

b591926c3be8
dfae19ce8dab
53403bd6b0fb
c8a297d46ea9
669baaee9af7
b591926c3be8
dfae19ce8dab
53403bd6b0fb
c8a297d46ea9
669baaee9af7


Now to start our app again via **docker-compose up**.

In [36]:
!docker-compose -f /ffdd/app/app.yml up -d

/usr/local/Cellar/fig/1.3.3/libexec/vendor/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
Creating app_web_1...


...And look at our composed containers via **docker-compose ps**.

In [37]:
!docker-compose -f /ffdd/app/app.yml ps

/usr/local/Cellar/fig/1.3.3/libexec/vendor/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
  Name        Command     State         Ports        
----------------------------------------------------
app_web_1   /app/web.py   Up      0.0.0.0:80->80/tcp 


Our container looks lonely.

## flask & redis in yaml

Let's add a redis service to count our page views.

Docker Best Practice: 1 process per container.

This means updating three files:
- app.yml
- web.py
- requirements.txt
<p>

Then, we'll need to rebuild our image.

And finally, we'll reload our application.

First, we add the redis service to the yaml.

In [44]:
app_yml = """
web:
  image: web:20150629-1800
  links:
   - redis
  ports:
   - '80:80'
  command: /app/web.py
redis:
  image: redis:3.0.2
"""
!echo "$app_yml" > /ffdd/app/app.yml

Don't forget the 'link'.

Then we update web.py to use redis.

In [45]:
web_py = """#!/usr/bin/env python
from flask import Flask
from redis import Redis

app = Flask(__name__)
redis = Redis(host='redis', port=6379)

@app.route('/')
def hello():
    redis.incr('hits')
    hits = int(redis.get('hits'))
    return 'Hello, world. Page hits: {hits}'.format(hits=hits)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=80, debug=True)
"""
!echo "$web_py" > /ffdd/app/web.py

Next, we update our requirements.txt to include redis.

In [46]:
web_requirements_txt = """
Flask==0.10.1
redis==2.10.3
"""
!echo "$web_requirements_txt" > /ffdd/app/requirements.txt

Always pin your versions!

Finally, we rebuild our image.

In [47]:
!time docker build --tag $web_image -f /ffdd/app/web_dockerfile /ffdd/app | tail -n 3   

 ---> 119dbce6214d
Removing intermediate container 78fd341b9b27
Successfully built 119dbce6214d

real	0m9.409s
user	0m0.173s
sys	0m0.011s


Now we can spin up our new redis + flask app.

In [48]:
!docker-compose -f /ffdd/app/app.yml up -d

/usr/local/Cellar/fig/1.3.3/libexec/vendor/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
Recreating app_redis_1...
Recreating app_web_1...


All those steps may seem reasonable when adding a new service.

But if our run-time dependencies don't change, we shouldn't need to rebuild our image.

As we've bundled our web.py into our image, we need to rebuild our image every time it changes. Ew.

In [51]:
!touch /ffdd/app/web.py
!time docker build --tag $web_image -f /ffdd/app/web_dockerfile /ffdd/app | tail -n 3  

 ---> Using cache
 ---> 283ba68b4c3e
Successfully built 283ba68b4c3e

real	0m0.278s
user	0m0.131s
sys	0m0.008s


We can't wait that long for each code update.

We can speed up our **image**'s **build** by reording our **Dockerfile**.

## reordering our dockerfile

In [53]:
# The old order:
!cat /ffdd/app/web_dockerfile


FROM python:3.4

RUN mkdir /app
ADD web.py /app/web.py

ADD requirements.txt /requirements.txt

RUN pip install -r /requirements.txt

EXPOSE 80



In [54]:
#    Our new order:
web_dockerfile =   """
FROM python:3.4
EXPOSE 80 

ADD requirements.txt /requirements.txt
RUN pip install -r /requirements.txt

RUN mkdir /app
ADD web.py /app/web.py
"""
!echo "$web_dockerfile" > /ffdd/app/web_dockerfile

Did we speed it up?

In [56]:
!touch /ffdd/app/web.py
!time docker build --tag $web_image -f /ffdd/app/web_dockerfile /ffdd/app | tail -n 3  

 ---> f496831ab573
Removing intermediate container b866a90db32b
Successfully built f496831ab573

real	0m0.640s
user	0m0.143s
sys	0m0.008s


## decoupling web.py and our web **image**

How to avoid rebuilding our **image** when changing web.py?

We could mount our source code as a host volume.

Let's start by decoupling web.py from our web **image**.

This will require removing it from our web **image**'s **Dockerfile**.

In [57]:
web_dockerfile = """
FROM python:3.4
EXPOSE 80

ADD requirements.txt /requirements.txt
RUN pip install -r /requirements.txt
"""
!echo "$web_dockerfile" > /ffdd/app/web_dockerfile

And placing it its own **Dockerfile**.

In [58]:
source_dockerfile = """
# I wish I were scratch!
FROM busybox:buildroot-2013.08.1
COPY . /app
VOLUME /app
"""
!echo "$source_dockerfile" > /ffdd/app/source_dockerfile

We'll need to include this new source service in our YAML.

In [59]:
app_yml = """
source:
  build: .
web:
  image: web:20150629-1800
  volumes_from:
   - source
  links:
   - redis
  ports:
   - '80:80'
  command: /app/web.py
redis:
  image: redis:3.0.2
"""
!echo "$app_yml" > /ffdd/app/app.yml

Note the new _build_ and _volumes_from_ directives.

Let's run it.

In [60]:
!docker-compose -f /ffdd/app/app.yml up -d

/usr/local/Cellar/fig/1.3.3/libexec/vendor/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
Creating app_source_1...
Recreating app_redis_1...
Recreating app_web_1...


In [61]:
!mv /ffdd/app/source_dockerfile /ffdd/app/Dockerfile

## decoupling web.py and web_image

Okay, let's make a minor change to web.py.

In [62]:
web_py = """#!/usr/bin/env python
from flask import Flask
from redis import Redis

app = Flask(__name__)
redis = Redis(host='redis', port=6379)

@app.route('/')
def hiya():
    redis.incr('hits')
    hits = int(redis.get('hits'))
    return 'Hiya, world! Page hits: {hits}'.format(hits=hits)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=80, debug=True)
"""
!echo "$web_py" > /ffdd/app/web.py

And run it.

In [63]:
!docker-compose -f /ffdd/app/app.yml up -d

/usr/local/Cellar/fig/1.3.3/libexec/vendor/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
Recreating app_source_1...
Recreating app_redis_1...
Recreating app_web_1...


Why doesn't the app reflect our change?

In [64]:
!docker-compose -f /ffdd/app/app.yml up -d

/usr/local/Cellar/fig/1.3.3/libexec/vendor/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning: A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.
  InsecurePlatformWarning
Recreating app_source_1...
Recreating app_redis_1...
Recreating app_web_1...


## Don't expect too much from **docker-compose up**.

It only executes a **build** instruction _once_.
<p>
It re-uses any previouly built (and potentially stale) images.

In order to get an updated image, we must either explicitly call **docker-compose build** or remove the old image.

## Don't put any more in your image than you must.

Changing web.py shouldn't mandate rebuilding web **image**.

## Don't forget volumes.
  

By default, **docker rm** does _not_ remove associated volumes.
- **docker-compose** replaces existing containers and re-uses volumes.
- These orphaned volumes can add up.
- Use **docker rm** -v or --volume to remove the container's volumes.

# Amazon Web Services

## TLAs (Three Letter Acronyms)

AWS (Amazon Web Services)

EC2 (Elastic Compute Cloud)

ECS (EC2 Container Service)

ELB (Elastic Load Balancing)

AMI (Amazon Machine Image)

IAM (Identity and Access Management)

Security Groups<p>
Launch Configuration<p>
Auto Scaling Groups<p>

Need Task Definitions
    Analogous to a docker-compose.yml file
    Need a docker-compose.yml to JSON converter?
    Introducing [container-transform](http://github.com/ambitioninc/container-transform/)
    Need mem_limit and cpu_shares

# CircleCI

In [ ]:
machine:
  services:
    - docker
  pre:
    - sudo pip install -U docker-compose
  environment:
    DOCKER_HUB_USER: gJigsaw
    DOCKER_HUB_EMAIL: CircleCI@JasonGreen.Name
  post:
    - docker info

dependencies:
  override:
    - docker login -e $DOCKER_HUB_EMAIL -u $DOCKER_HUB_USER -p $DOCKER_HUB_PASSWORD
    - sudo ln -sf /home/ubuntu/repository_name /ffdd
    - /ffdd/get_images.sh

test:
  pre:
    - ls -al /aver && cd /aver && ls -al
  override:
    - /aver/informatics/run_unit_tests.sh
    - /aver/informatics/run_integration_tests.sh

deployment:
  staging:
    branch: master
    heroku:
      appname: foo-bar-123

notify:
  webhooks:
    - url: https://someurl.com/hooks/circle

In [ ]:
dependencies:
  cache_directories:
    - *docker_cache_directory
  environment:
    DOCKER_HUB_USER: gJigsaw
    DOCKER_HUB_EMAIL: CircleCI@JasonGreen.Name
    DOCKER_CACHE_DIRECTORY: &docker_cache_directory '/home/ubuntu/docker_cache'
  override:
    - /ffdd/get_images.sh

In [ ]:
  pre:
    - sudo curl -L https://github.com/docker/machine/releases/download/v0.2.0/docker-machine_darwin-amd64 > /usr/local/bin/docker-machine && sudo chmod +x /usr/local/bin/docker-machine
  post:
    - docker-machine --version
    - docker-machine create --url=tcp://54.88.9.37:2376 circle-ci


In [ ]:
#!/bin/bash

set -e
set -x

mkdir -p $DOCKER_CACHE_DIRECTORY/aver

for IMAGE in $IMAGES; do
    TARBALL_PATH=$DOCKER_CACHE_DIRECTORY/$IMAGE
    if [[ -e $TARBALL_PATH ]]; then
        docker load -i $TARBALL_PATH;
    else
        docker pull --all-tags=false $IMAGE
        docker save $IMAGE > $TARBALL_PATH;
    fi
done

# I've Nothing else.

gJigsaw  <sub><sub>gMail, GitHub, <sub> and **very** infrequently Twitter</sub></sub></sub>